In [10]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

### 1. 웹 페이지 크롤링

fetch(url): 매개변수로 지정한 URL의 웹 페이지를 추출

In [11]:
def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출
    웹 페이지의 Content-Type 헤더를 통해 인코딩 형식 확인
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식 추출
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열 디코딩
    html = f.read().decode(encoding)
    return html

### 2. 스크래핑

scrape(html): 매개변수로 html을 받고, 정규 표현식 / html 파서를 사용해 HTML에서 도서 정보 추출

In [12]:
def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목 추출
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append(pd.DataFrame({'url': [url], 'title': [title]}))
    return pd.concat(books)

### 3. 데이터 저장 (CSV / JSON / DBMS)

save(db_path, books): 매개변수로 books라는 도서 목록을 받고, db_path로 지정된 SQLite DB에 저장

In [13]:
def save(db_path, books):
    with sqlite3.connect(os.path.join('.', db_path)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            books.to_sql(name = 'BOOKS_INFO', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
    
    query = 'SELECT * FROM BOOKS_INFO'
    df = pd.read_sql(query, con = con)
    return df

In [14]:
html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [15]:
df = scrape(html)
df.reset_index(drop=True, inplace=True)
df2 = save('books.db', df)
df2

,url,title
0,http://www.hanbit.co.kr/store/books/look.php?p...,청소년 인문학 수업 - 1권
1,http://www.hanbit.co.kr/store/books/look.php?p...,청소년 인문학 수업 - 2권
2,http://www.hanbit.co.kr/store/books/look.php?p...,처음 시작하는 파이썬(2판)
3,http://www.hanbit.co.kr/store/books/look.php?p...,퀀트 전략을 위한 인공지능 트레이딩
4,http://www.hanbit.co.kr/store/books/look.php?p...,처음 시작하는 딥러닝
...,...,...
145,http://www.hanbit.co.kr/store/books/look.php?p...,"IT CookBook, 컴퓨터 구조와 원리 3.0"
146,http://www.hanbit.co.kr/store/books/look.php?p...,"IT CookBook, 최신 기술 동향으로 알아보는 ICT와 4차 산업혁명"
147,http://www.hanbit.co.kr/store/books/look.php?p...,초보 판매자가 빅파워셀러로 거듭나는 네이버 스마트스토어 마케팅 시작하기
148,http://www.hanbit.co.kr/store/books/look.php?p...,"STEM CookBook, 한 걸음씩 알아가는 선형대수학"
